# Hello World

將程式碼和 Markdown 文件，放在一起，邊看說明邊執行 😀

先來個輸出 Hello World 吧


In [ ]:
Console.WriteLine("Hello World");

預設載入以下套件參考

- .NETStandard.Library
- Microsoft.AspNetCore.Html.Abstractions
- Microsoft.DotNet.Interactive
- Microsoft.DotNet.Interactive.Formatting

預設使用以下 namespace

- `System`
- `System.Collections`
- `System.Collections.Generic`
- `System.Ling`
- `System.Threading.Tasks`
- `System.Text`
- `Microsoft.DotNet.Interactive`
- `Microsoft.DotNet.Interactive.Formatting`

## .NET Notebook 的 C# 語法

- .NET Notebook 使用修改版的 C# Script 語法
- 也就是 `.csx` 腳本的格式，跟平常寫的 C# 不一樣但也差不多
- http://scriptcs.net/



# 使用其他 NuGet 套件

預設的功能往往不足以使用，我們可以透過 NuGet 來擴充 API


In [ ]:
// 設定 NuGet.org 的套件來源
#i "nuget:https://api.nuget.org/v3/index.json"
// 使用本機環境下的 NuGet 套件來源
// #i "nuget:C:\myorg\mypackage\src\bin\Release"

// 還原指定的 NuGet 套件
// 指定最新版
#r "nuget:Newtonsoft.Json"

// 指定版本號
// #r "nuget:Newtonsoft.Json,13.0.1"

// 指定最新 pre-release 版本
// #r "nuget:Newtonsoft.Json,*-*"


In [ ]:
using Newtonsoft.Json;

var program = new { Name = ".NET Notebook", Year = 2021 };
var support = new []{ "Jupyter", "nteract", "Visual Studio Code" };
Console.WriteLine(JsonConvert.SerializeObject(program, Formatting.Indented));
Console.WriteLine(JsonConvert.SerializeObject(support));


除了使用 C# 環境中常用的 `Console.WriteLine()` 來輸出結果之外，還可以使用 .NET Notebook 內建的 `display()` 功能來

In [ ]:
display(program);
display(support);

當想要查看所有頂層變數的資訊時，可以使用以下兩個 Magic Command

- `#!who` 顯示頂層變數的名稱
- `#!whos` 顯示頂層變數的名稱、類型和值


In [ ]:
#!who
#!whos

# Magic Command

在行的開頭，並以 `#!` 或 `#` 作為前綴的寫法，可被視為 Magic Command

目的在於將複雜的行為封裝成單一指令，用更快捷的方式執行

基本上可分為以下兩種類型

- 通用型
  - 宣告類 eg. `#!markdown`、`#!csharp`...
  - 功能類 eg. `#!about`、`#!lsmagic`、`#!time`...
- 特定語言專用


In [ ]:
#!html

<p>Hello World</p>

In [ ]:
#!about

In [ ]:
#!lsmagic

In [ ]:
#!time
System.Threading.Thread.Sleep(1000);

## #!share

.NET Interactive 提供 C#、F# 和 PowerShell 的 Subkernal 共享變數的功能

In [ ]:
#!csharp
var speaker = new { Name = "Poy Chang", Age = 37 };

speaker

In [ ]:
#!pwsh
#!share --from csharp speaker

ConvertTo-Json $speaker

## #!value

我們也可以單純的透過 `#!value` 命令將以下兩種來源的值轉換成變數

- 直接給予資料值
- 從指定的檔案中讀取

In [ ]:
#!value --name speakerJson
{ "Name": "Poy Chang", "Age": 37 }

In [ ]:
#!share speakerJson --from value

class Speaker {
    public string Name { get; set; }
    public int Age { get; set; }
}

var speaker = System.Text.Json.JsonSerializer.Deserialize<Speaker>(speakerJson);
display(speaker);


In [ ]:
#!value --from-file speaker-data.json --name speakerData

In [ ]:
#!share speakerData --from value

display(speakerData);

在設計 Notebook 的時候，有時候會想要讓使用這自行輸入某些設定值，當然使用者可以直接修改程式，不過 `#!value` 搭配 `--from-value` 可以提供一種彈跳輸入框的方式，讓使用者可以更直覺的輸入資料

- `@input`
- `@password`

>目前還是隱藏版的功能，沒有文件，但有單元測試 [InputsWithinMagicCommandsTests.cs#L66](https://github.com/dotnet/interactive/blob/main/src/Microsoft.DotNet.Interactive.Tests/InputsWithinMagicCommandsTests.cs#L66)

In [ ]:
#!value --from-value @input:myName --name myName

In [ ]:
#!share --from value myName

myName

poy


# 呼叫 REST API

用最簡單的方式來呼叫 REST API

In [ ]:
using System.Net.Http;

var contents = await new HttpClient().GetStringAsync("https://jsonplaceholder.typicode.com/todos/1");

display(contents);

# 連結資料庫


In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

In [ ]:
#!connect mssql --create-dbcontext --kernel-name adventureworks "Server=localhost\SQLEXPRESS;Database=AdventureWorksLT2019;Trusted_Connection=True;"

In [ ]:
#!sql-adventureworks

select * from SalesLT.Product

In [ ]:
adventureworks.Product.ExploreWithSandDance()

# Sand Dance

[SandDance](https://microsoft.github.io/SandDance/) 是微軟研究院推出的數據可視化工具

提供使用者和 3D 訊息圖表進行互動，並且可以用不同的角度來呈現分析結果，讓使用者藉由可視化的方式直觀的接受數據訊息，幫助我們找到數據中潛在的特徵


In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

In [ ]:
using System.IO;
using System.Net.Http;

string housingPath = "housing.csv";

if (!File.Exists(housingPath))
{
    var contents = await new HttpClient()
        .GetStringAsync("https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv");
        
    File.WriteAllText("housing.csv", contents);
}

In [ ]:
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;

var housingData = DataFrame.LoadCsv(housingPath);

In [ ]:
housingData.ExploreWithSandDance().Display();